In [1]:
###########################
### MODULES NECESSAIRES ###
###########################


# Import the necessary libraries
import numpy as np
import pandas as pd
import math
import fonctionsSupervisedLearning2 as fsl2

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from auxFonctions import AminoAcid

/var/folders/wp/4cr2jl295bs4z4czxy_p6w5m0000gn/T/ipykernel_21009/86324599.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/mathiasperez/Documents/GitHub/442-2-Protein-cleavage/fonctionsSupervisedLearning2.py:132: SyntaxWarning: invalid escape sequence '\ '
  '''


In [2]:
# Read data from a file into a list of entries
with open('data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')

In [3]:
##############################
## RECUPERATION DES DONNÉES ##
##############################

# Process each entry
processed_entries = [fsl2.process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))

# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

## Traitement des données

Put words in vector and vice versa

In [4]:
df_exploitable = fsl2.convert_df_to_vectors2(df)

In [21]:
n = 15
p,q = 13, 2  # p+q = 15

In [22]:
# Create a DataFrame to store, for each primary structure, the neihborhood of the cleavage site
# The neighborhood is defined as the word of length p+q starting p letters before the cleavage site
correct_neighborhood = pd.Series()
for i, seq in amino_acid_seq.items():
    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p:cleavage_site_position[i]+q])

In [23]:
# Create a DataFrame to store, for each primary structure, a sequence that is not the neighborhood of the cleavage site
incorrect_neighborhood = pd.Series()
decalage  = [1,2,3,4,5, -1,-2,-3,-4, -5]
for i, seq in amino_acid_seq.items():
    dec = np.random.choice(decalage)
    dec = 0 if cleavage_site_position[i]-13 - dec < 0 else dec
    incorrect_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p-dec:cleavage_site_position[i]+q-dec])

In [24]:
#print(correct_neighborhood.get(4))
print(fsl2.word_to_vector(incorrect_neighborhood.get(52)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Train the models

In [6]:
X_train, X_test, bool_train, bool_test = fsl2.test_train_split_random_pos2(df_exploitable, 15, 13, random_state=42)

count_true = 0
count_all = 0
for t in bool_train:
    count_all += 1
    if t:
        count_true += 1

print(count_true)
print(count_all)
print(bool_train[0])

TypeError: extract_random_subsequence2() missing 1 required positional argument: 'row'

In [10]:
#svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable, random_state=42, nb_letters = 26, kernel_neighbor= 'rbf',C_nei = 1)


C_list = [0.1,1,2, 5]
kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']
p_list = [3, 5, 7, 9, 11, 13, 15]
q_list = [2, 4, 6, 8]
results = []

for C_nei in C_list:
    for kernel_nei in kernel_list:
        for p in p_list:
            for q in q_list:
                n = p+q
                correct_neighborhood = pd.Series()
                for i, seq in amino_acid_seq.items():   
                    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p:cleavage_site_position[i]+q])

                incorrect_neighborhood = pd.Series()
                decalage  = [1,2,3,4,5, -1,-2,-3,-4, -5]
                for i, seq in amino_acid_seq.items():
                    dec = np.random.choice(decalage)
                    dec = 0 if cleavage_site_position[i]-13 - dec < 0 else dec
                    incorrect_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p-dec:cleavage_site_position[i]+q-dec])

                svm_model_nei, accuracy_nei = fsl2.create_model2(n, df_exploitable, kernel_nei, C_nei)

                count_fn = 0
                for i in range(0, len(correct_neighborhood)):
                    x = np.array([fsl2.word_to_vector(correct_neighborhood.get(i))])
                    test = svm_model_nei.predict(x)
                    if (test == 0):
                        count_fn += 1
                count_fp = 0
                for i in range(0, len(incorrect_neighborhood)):
                    x = np.array([fsl2.word_to_vector(incorrect_neighborhood.get(i))])
                    test = svm_model_nei.predict(x)
                    if (test == 1):
                        count_fp += 1
            
                results.append((C_nei, kernel_nei, p, q, accuracy_nei, count_fn/len(correct_neighborhood), count_fp/len(incorrect_neighborhood)))

results_df = pd.DataFrame(results, columns=['C_nei','kernel_nei', 'p', 'q', 'accuracy_nei', 'false negative', 'false positive'])
results_df.to_csv('results2.csv', index=True)


In [11]:
best_results = results_df.loc[(results_df['accuracy_nei']).idxmax()]
print(best_results)

C_nei               1.0
kernel_nei         poly
n                    19
accuracy_nei    0.91844
Name: 23, dtype: object


In [12]:
svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable,'rbf', 1.0)

In [13]:
print(accuracy_nei)

0.8687943262411347


In [14]:
count = 0
for i in range(0, len(correct_neighborhood)):
    X = fsl2.word_to_vector(df["Primary Structure"].get(i))
    if fsl2.find_cleavage2(X,svm_model_nei,15):
        count += 1

print(count)


1404


In [15]:
count_fn = 0
for i in range(0, len(correct_neighborhood)):
    test = fsl2.is_neighborhood2(correct_neighborhood.get(i), svm_model_nei, 15)
    if not(test):
        count_fn += 1

print("Nombre de faux négatifs :" + str(count_fn))

count_fp = 0
for i in range(0, len(incorrect_neighborhood)):
    try : 
        test = fsl2.is_neighborhood2(incorrect_neighborhood.get(i), svm_model_nei, 15)
        if test:
            count_fp += 1
    except:
        print("Erreur" + str(i))
        #print(incorrect_neighborhood.get(i))


print("Nombre de faux positifs :" + str(count_fp))
print("Nombre total :" + str(len(correct_neighborhood)))




Nombre de faux négatifs :126
Nombre de faux positifs :460
Nombre total :1408
